In [1]:
%reset -sf

In [2]:
# LOAD THIS BEFORE START

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
# import scipy
# import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [87]:
# with open("/Users/hkmac/Downloads/input.txt") as f:
#     srr = f.read().strip()

In [56]:
srr = """
""".strip()

In [57]:
srr = """
30373
25512
65332
33549
35390
""".strip()

In [58]:
matrix = [[int(x) for x in row] for row in srr.split("\n")]

In [59]:
n = len(matrix)
m = len(matrix[0])

In [60]:
maxres = 0
for i in range(n):
    for j in range(m):
        a,b,c,d = 0,0,0,0
        seen = False
        for x in list(range(i))[::-1]:
            if matrix[x][j] >= matrix[i][j]:
                a += 1
                break
            a += 1
        else:
            seen = True

        for x in range(i+1,n):
            if matrix[x][j] >= matrix[i][j]:
                b += 1
                break
            b += 1
        else:
            seen = True

        for y in list(range(j))[::-1]:
            if matrix[i][y] >= matrix[i][j]:
                c += 1
                break
            c += 1
        else:
            seen = True

        for y in range(j+1,m):
            if matrix[i][y] >= matrix[i][j]:
                d += 1
                break
            d += 1
        else:
            seen = True
        
#         print(a,b,c,d)
        res = a*b*c*d
        maxres = max(maxres, res)
            

In [61]:
maxres

8

In [55]:
res = 0

In [91]:
# sum(sorted(brr)[-3:])

In [92]:
def test(arr):
    res = 0
    return res